<a href="https://colab.research.google.com/github/Ravgeer/sparkscon2024/blob/main/ComputerVision_Sparkscon2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Connect G-Drive, copy data and unzip**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/adesso/wm21.zip /content

In [ ]:
!unzip -q /content/wm21.zip -d /content

In [1]:
!git clone https://github.com/Ravgeer/sparkscon2024

Cloning into 'sparkscon2024'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (7/7), done.


# **2. Install YOLO and it's pretrained models**

In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-324-gbb27bf78 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.3/201.2 GB disk)


In [ ]:
from utils.downloads import attempt_download
p5 = ['n','s','l','m','x']

cls = [f'{x}-cls' for x in p5]

for x in cls:
  attempt_download(f'weights/yolov5{x}.pt')

# **3.a. Object recognition training and test detection**

In [ ]:
!python train.py --img 1200 --batch 8 --epochs 200 --data /content/wm_recognition/config/coco128.yaml --weights yolov5l.pt --cache

In [ ]:
!python detect.py --source /content/wm_recognition/images/test/beko2.jpg --weights runs/train/exp3/weights/best.pt --save-txt --img 1200 --conf 0.50


# **3.b. Image classification training and test detection**







In [ ]:
!python -W ignore classify/train.py --img 640 --batch 16 --epochs 50 --data  /content/wm_classification --model yolov5l-cls.pt

2024-05-14 09:06:10.906607: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 09:06:10.906662: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 09:06:10.908073: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
classify/train: model=yolov5l-cls.pt, data=/content/wm_classification, epochs=50, batch_size=16, imgsz=640, nosave=False, cache=None, device=, workers=8, project=runs/train-cls, name=exp, exist_ok=False, pretrained=True, optimizer=Adam, lr0=0.001, decay=5e-05, label_smoothing=0.1, cutoff=None, dropout=None, verbose=False, seed=0, local_rank=-1
github: up to date 

In [ ]:
!python classify/predict.py --weights runs/train-cls/exp2/weights/best.pt --source /content/wm_recognition/images/train/a1c881e6-Beko_WMC91464ST1_15.jpg --save-txt

Vishal's trails

In [ ]:
!python -W ignore classify/train.py --img 640 --batch 16 --epochs 10 --data  /content/wm_classification --model yolov5x-cls.pt

# **4. Export model**

In [ ]:
!python export.py --weights runs/train-cls/exp9/weights/best.pt --include onnx

In [ ]:
!python export.py --weights runs/train/exp/weights/best.pt --include torchscript onnx saved_model

# **5. Load Model**

In [ ]:
from ultralytics import YOLO

# Load a model
# model = YOLO('yolov8n.pt')  # load an official model
model = YOLO('/content/yolov5/runs/train/exp/weights/best.pt')  # load a custom trained model

# Export the model
model.export(format='onnx')

# **Archive**

In [ ]:
!mkdir /content/washing_machines/images/train
!mkdir /content/washing_machines/images/val

!mv /content/washing_machines/images/{1..50}.jpg.jpg /content/washing_machines/images/train
!mv /content/washing_machines/images/{51..77}.jpg.jpg /content/washing_machines/images/val

!mkdir /content/washing_machines/labels/train
!mkdir /content/washing_machines/labels/val

!mv /content/washing_machines/labels/{1..50}.jpg.txt /content/washing_machines/labels/train
!mv /content/washing_machines/labels/{51..77}.jpg.txt /content/washing_machines/labels/val

In [ ]:
!zip -r /content/drive/MyDrive/model_classification21.zip /content/yolov5/runs/train-cls/exp9

In [ ]:
!cp /content/model.zip /content/drive/MyDrive

In [ ]:
!rm -rf /content/wm_classification/